In [1]:
%run stdPackages.ipynb

# Read database

In [2]:
from pyDbs import read, readSets, broadcast
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['HourlyVariation'],
        'scalars': ['Scalars']}
db = read.dbFromWB(os.path.join(d['data'],'mBasicPHLarge.xlsx'), kwargs)
readSets(db)

This model takes roughly 30 seconds to solve. This primarily comes from: 
1. ```self.initBlocks``` (7-8 seconds)
2. ```self.compileParameters``` (5 seconds)
3. ```self.getDenseArgs``` (6 seconds)
4. ```self.postSolve``` (roughly 6 seconds)

Solution time requires roughly 1-2 seconds.

In [3]:
import time
times = {}
t0 = time.time()
# Initialize model:
m = mBasicPH.mSimple(db) # initialize using the database 'db' from above.
times['setup'] = time.time()-t0
# Initialize blocks:
m.preSolve()
m.initBlocks()
times['initBlocks'] = time.time()-t0-sum(times.values())
# Compile model:
m.blocks.compileParameters()
times['compileParameters'] = time.time()-t0-sum(times.values())
m.blocks.settingsFromCompiled()
times['settingsFromCompiled'] = time.time()-t0-sum(times.values())
m.blocks.inferGlobalDomains()
times['inferGlobalDomains'] = time.time()-t0-sum(times.values())
m.blocks.getDenseArgs()
times['getDenseArgs'] = time.time()-t0-sum(times.values())
args = m.blocks.lp_args
times['lp_args'] = time.time()-t0-sum(times.values())
sol = optimize.linprog(method = m.method, **args)
times['sol'] = time.time()-t0-sum(times.values())
m.postSolve(sol)
times['postSolve'] = time.time()-t0-sum(times.values())
times

{'setup': 0.3298301696777344,
 'initBlocks': 7.508715629577637,
 'compileParameters': 5.969946622848511,
 'settingsFromCompiled': 0.001001596450805664,
 'inferGlobalDomains': 3.427182674407959,
 'getDenseArgs': 9.77975583076477,
 'lp_args': 2.4429104328155518,
 'sol': 2.510622262954712,
 'postSolve': 5.663256883621216}